### Importing dependancies

In [1]:
import numpy as np
import requests
import pandas as pd
from datetime import datetime, timedelta

#### Setting url destination API and making request

In [2]:
# Define the API endpoint URL
url = "https://www.reddit.com/r/depressionregimens/top/.json?limit=100&t=month"

# Add user agent header to avoid 429 error
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Make the API request
response = requests.get(url, headers=headers)

# Check if the response was successful
if response.status_code != 200:
    raise ValueError("Failed to fetch posts: {}".format(response.status_code))

#### Parsing JSON response, storing retrieved data in 'posts_data'

In [3]:
data = response.json()
posts_data = []
for post in data["data"]["children"]:
    post_title = post["data"]["title"]
    post_body = post["data"]["selftext"]
    user = post["data"]["author"]
    posts_data.append({"user": user, "post_title": post_title, "post_body": post_body})

#### Saving posts_data as data frame 'df'

In [4]:
df = pd.DataFrame(posts_data)
df.to_excel('C:/Users/ryan_/OneDrive/School/Pharma/coding/data.xlsx', index=False)

#### Importing Natural Language Toolkit to do detailed semantic analysis

In [6]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

#downloadibf the necessary resources for the SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Initialize SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ryan_\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


#### Defining sentiment analysis function

In [7]:
def get_sentiment_scores(text):
    scores = sia.polarity_scores(text)
    return {'pos': scores['pos'], 'neg': scores['neg'], 'neu': scores['neu'], 'compound': scores['compound']}

#### Applying function and storing scores to original df in new columns

In [8]:
df[['pos', 'neg', 'neu', 'compound']] = df['post_title'].apply(lambda x: pd.Series(get_sentiment_scores(x)))

                   user                                         post_title  \
0  Evening_Attention_45   Does anyone else get completely stuck/paralyzed?   
1                quat1e  I tried group CBT about 8 years ago, and every...   
2  Responsible-Dish-977  Exhaustive list of genes that affect Mental He...   
3       Liberated051816  Study: Brain Serotonin Release Is Reduced In P...   
4         spyderspyders  Associations between Melatonin, Neuroinflammat...   

                                           post_body  pos    neg    neu  \
0  I find myself getting so stuck that I can’t ev...  0.0  0.000  1.000   
1  I have been doing one-to-one CBT for a few wee...  0.0  0.083  0.917   
2  This was one of the longest and most labor int...  0.0  0.091  0.909   
3  This study used a new and more direct method t...  0.0  0.291  0.709   
4                                                     0.0  0.291  0.709   

   compound  
0    0.0000  
1   -0.2263  
2   -0.1280  
3   -0.5719  
4   -0.571

OpenAI API Key

#### Importing OpenAI API

In [9]:
import openai
with open('openAI_apiKey.txt', 'r') as f:
    api_key = f.read().strip()
openai.api_key = api_key

### Getting drug names/treatments discussed in the post body

#### Defining function for getting drug name (retrieve_drugName)

In [10]:
def retrieve_drugName(text):
    prompt = (
        f"I want you to act as expert pharmaceutical research assistant. Identify the names of the drug or treatment that is the main focus of the following text. Your response must only be the names of drugs. If there are multiple drugs, list them in the order they are mentioned. If you are not sure, reply with 'not sure'. If it is not clear what drug is being discussed, reply 'N/A'. Do not respond with anything other then the names of drugs.: {text}\n")

    # Make API call to OpenAI
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.1,
    )

    # Extract drug names from API response
    output = response.choices[0].text.strip()
    
    # Return output
    return output

    drug_names = [word for word in output.split() if word.istitle()]

    # Return drug names (if any)
    return drug_names[0] if drug_names else None

In [11]:
df['drug_name'] = df['post_body'].apply(retrieve_drugName)

In [12]:
df['drug_name_title'] = df['post_title'].apply(retrieve_drugName)

#### Filtering out any values where the drug name was not assessed with OpenAI

In [13]:
df_filtered = df[~(((df['drug_name'].str.lower() == 'n/a') | (df['drug_name'].str.lower() == 'not sure') | (df['drug_name'].str.lower() == 'not sure.')) & ((df['drug_name_title'].str.lower() == 'n/a') | (df['drug_name_title'].str.lower() == 'not sure') | (df['drug_name_title'].str.lower() == 'not sure.')))]

In [15]:
df_filtered = df_filtered.rename(columns={'drug_name': 'drug_name_body'})
print(df_filtered.columns)

Index(['user', 'post_title', 'post_body', 'pos', 'neg', 'neu', 'compound',
       'drug_name_body', 'drug_name_title'],
      dtype='object')


### Creating single drug name from drug_name_title or drug_name_body

In [16]:
def get_valid_value(row):
    valid_values = [value.lower() for value in ['n/a', 'not sure', 'not sure.']]
    if row['drug_name_title'].lower() not in valid_values:
        return row['drug_name_title']
    elif row['drug_name_body'].lower() not in valid_values:
        return row['drug_name_body']
    else:
        return pd.np.nan

In [17]:
# Apply the lambda function to create the new column
df_filtered['drug_name'] = df_filtered.apply(lambda row: get_valid_value(row), axis=1)

                    user                                         post_title  \
4          spyderspyders  Associations between Melatonin, Neuroinflammat...   
7          purplebadger9  On top of my daily Lexapro, weekly Spravato, a...   
10       depressedthotty  Am I just fkin broken? Why isn’t this working ...   
12    allthethrowaway420  Have antidepressants actually ever worked for ...   
13           iLiveOnFear  My experience with beating Depression (Solutio...   
21            milkman246           Lexapro is working and it feels so great   
22        CuriousBetsy69             do psychiatric drugs damage the brain?   
24            farhad_666                              Proviron, Holy shit!!   
25  Ill_Championship_571                     Accuracy of GeneSight testing?   
26             nirman248  Why most pschyiatrist can't treat treatment re...   

                                            post_body    pos    neg    neu  \
4                                                   

### Identifying generic drug name equivalants

In [18]:
# Define function to classify drug names
def generic_drug(drug_name):
    prompt = (
        f"I want you to act as expert pharmaceutical assistant. Identify the generic drug name from the drug in the prompt provided and provide it as your response - just the generic name, do not also include the brand name. If there are multiple drugs, list their classifications in the order they are mentioned. You response must only consist of the generic drug name for the drug given in the prompt. If there is no clear drug being discussed, reply 'N/A'. Do not respond with any brand names - note that Lexapro and Zoloft are brand names, find their generic equivanlent if prompted with them.: {drug_name}\n"
    )

    # Make API call to OpenAI
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.1,
    )

    # Extract drug class from API response
    output = response.choices[0].text.strip()
    drug_generic = output.split('\n')[0]

    # Return drug class
    return drug_generic


In [19]:
df_filtered['generic_name'] = df_filtered['drug_name'].apply(generic_drug)

In [20]:
# print(df_filtered2.head(10))
df_filtered2 = df_filtered.dropna(subset=['drug_name'])

#### Filtering out n/a values

In [22]:
df_filtered3 = df_filtered2.dropna(subset=['generic_name'], axis=0)
df_filtered4 = df_filtered3[df_filtered3['generic_name'] != 'N/A']

#### Creating count of unique mentions

In [24]:
drug_count = {}

for index, row in df_filtered4.iterrows():
    drugs = row['generic_name'].split(", ")  # assuming drug names are separated by comma and space
    for drug in drugs:
        drug = drug.split(":")[-1].strip().lower()  # get the drug name after the colon (if any) and convert to lowercase
        if drug in drug_count:
            drug_count[drug] += 1
        else:
            drug_count[drug] = 1

#### Ranking drugs by count

In [25]:
# convert drug_count to a DataFrame and sort it by count
ranked_drugs = pd.DataFrame(drug_count.items(), columns=['genetic_name', 'count']).sort_values(by='count', ascending=False)

                                         genetic_name  count
0                                        escitalopram     15
4                                          sertraline      9
28                                         fluoxetine      6
27                                          bupropion      6
2                                                 n/a      6
13                                        venlafaxine      5
71                                        fluvoxamine      4
36                                         citalopram      4
8                                                          4
68                                       aripiprazole      4
22                                           ketamine      4
65                                          buspirone      3
16                                            lithium      3
23                                        amphetamine      3
35                                         paroxetine      2
74                      

In [26]:
ranked_drugs['genetic_name'].replace(['N/A', '', 'n/a'], pd.NaT, inplace=True)
ranked_drugs= ranked_drugs.dropna(subset=['genetic_name'])

In [27]:
total_count = ranked_drugs['count'].sum()
ranked_drugs['share'] = ranked_drugs['count'] / total_count

    genetic_name  count     share
0   escitalopram     15  0.096154
4     sertraline      9  0.057692
28    fluoxetine      6  0.038462
27     bupropion      6  0.038462
13   venlafaxine      5  0.032051
71   fluvoxamine      4  0.025641
36    citalopram      4  0.025641
68  aripiprazole      4  0.025641
22      ketamine      4  0.025641
65     buspirone      3  0.019231


### Creating drug specific data frames for each of the major competitors + ketamine 

In [29]:
escitalopram_df = pd.DataFrame(columns=df_filtered3.columns)

# Iterate through the rows of df_filtered3
for index, row in df_filtered3.iterrows():
    # Check if 'Bupropion' (case insensitive) is in the 'generic_name' column
    if 'escitalopram' in str(row['generic_name']).lower():
        # Append the row to bupropion_df
        escitalopram_df = escitalopram_df.append(row)

# Reset the index of bupropion_df
escitalopram_df = escitalopram_df.reset_index(drop=True)

C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\2837894275.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  escitalopram_df = escitalopram_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\2837894275.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  escitalopram_df = escitalopram_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\2837894275.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  escitalopram_df = escitalopram_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\2837894275.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  escitalopram_df = escitalopram_df.append(row)
C:\Users

In [30]:
bupropion_df = pd.DataFrame(columns=df_filtered3.columns)

# Iterate through the rows of df_filtered3
for index, row in df_filtered3.iterrows():
    # Check if 'Bupropion' (case insensitive) is in the 'generic_name' column
    if 'bupropion' in str(row['generic_name']).lower():
        # Append the row to bupropion_df
        bupropion_df = bupropion_df.append(row)

# Reset the index of bupropion_df
bupropion_df = bupropion_df.reset_index(drop=True)

C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\293645895.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bupropion_df = bupropion_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\293645895.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bupropion_df = bupropion_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\293645895.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bupropion_df = bupropion_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\293645895.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bupropion_df = bupropion_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ip

In [31]:
sertraline_df = pd.DataFrame(columns=df_filtered3.columns)

# Iterate through the rows of df_filtered3
for index, row in df_filtered3.iterrows():
    # Check if 'Bupropion' (case insensitive) is in the 'generic_name' column
    if 'sertraline' in str(row['generic_name']).lower():
        # Append the row to bupropion_df
        sertraline_df = sertraline_df.append(row)

# Reset the index of bupropion_df
sertraline_df = sertraline_df.reset_index(drop=True)

C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\315095891.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sertraline_df = sertraline_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\315095891.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sertraline_df = sertraline_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\315095891.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sertraline_df = sertraline_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\315095891.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sertraline_df = sertraline_df.append(row)
C:\Users\ryan_\AppData\Local

In [32]:
fluoxetine_df = pd.DataFrame(columns=df_filtered3.columns)

# Iterate through the rows of df_filtered3
for index, row in df_filtered3.iterrows():
    # Check if 'Bupropion' (case insensitive) is in the 'generic_name' column
    if 'fluoxetine' in str(row['generic_name']).lower():
        # Append the row to bupropion_df
        fluoxetine_df = fluoxetine_df.append(row)

# Reset the index of bupropion_df
fluoxetine_df = fluoxetine_df.reset_index(drop=True)

C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\622303249.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fluoxetine_df = fluoxetine_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\622303249.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fluoxetine_df = fluoxetine_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\622303249.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fluoxetine_df = fluoxetine_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\622303249.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fluoxetine_df = fluoxetine_df.append(row)
C:\Users\ryan_\AppData\Local

In [33]:
fluoxetine_df = pd.DataFrame(columns=df_filtered3.columns)

# Iterate through the rows of df_filtered3
for index, row in df_filtered3.iterrows():
    # Check if 'Bupropion' (case insensitive) is in the 'generic_name' column
    if 'fluoxetine' in str(row['generic_name']).lower():
        # Append the row to bupropion_df
        fluoxetine_df = fluoxetine_df.append(row)

# Reset the index of bupropion_df
fluoxetine_df = fluoxetine_df.reset_index(drop=True)

C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\622303249.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fluoxetine_df = fluoxetine_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\622303249.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fluoxetine_df = fluoxetine_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\622303249.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fluoxetine_df = fluoxetine_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\622303249.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fluoxetine_df = fluoxetine_df.append(row)
C:\Users\ryan_\AppData\Local

In [34]:
ketamine_df = pd.DataFrame(columns=df_filtered3.columns)

# Iterate through the rows of df_filtered3
for index, row in df_filtered3.iterrows():
    # Check if 'Bupropion' (case insensitive) is in the 'generic_name' column
    if 'ketamine' in str(row['generic_name']).lower():
        # Append the row to bupropion_df
        ketamine_df = ketamine_df.append(row)

# Reset the index of bupropion_df
ketamine_df = ketamine_df.reset_index(drop=True)

C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\1633849483.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ketamine_df = ketamine_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\1633849483.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ketamine_df = ketamine_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\1633849483.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ketamine_df = ketamine_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\1633849483.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ketamine_df = ketamine_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipyker

In [35]:
citalopram_df = pd.DataFrame(columns=df_filtered3.columns)

for index, row in df_filtered3.iterrows():
    if 'citalopram' in str(row['generic_name']).lower():
        # Append the row to bupropion_df
        citalopram_df = citalopram_df.append(row)
citalopram_df = citalopram_df.reset_index(drop=True)

C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\4038485475.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  citalopram_df = citalopram_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\4038485475.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  citalopram_df = citalopram_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\4038485475.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  citalopram_df = citalopram_df.append(row)
C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\4038485475.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  citalopram_df = citalopram_df.append(row)
C:\Users\ryan_\AppData\L

### Summarizing the posts

#### Defining function to summarize posts

In [36]:
def summarize_text(text):
    prompt = (
        f"I want you to act as natural language assistant specializing in pharmaceuticals. You will be given the body text of a reddit post, I want you to read the user's post and concisely summarize it to identify the user's primary complaint or concern. Your summary must be under 20 words. Do not start your response with 'the user is' or any variation of, just respond with the concern or complaint. Post: {text}\n")

    # Make API call to OpenAI
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.1,
    )

    # Extract first 5 words from API response
    output = response.choices[0].text.strip().split()
    summary = ' '.join(output)

    # Return summary
    return summary

#### Applying function to each of the drug specific data frames

In [37]:
escitalopram_df['post_summary'] = escitalopram_df['post_body'].apply(summarize_text)

In [38]:
bupropion_df['post_summary'] = bupropion_df['post_body'].apply(summarize_text)

In [39]:
sertraline_df['post_summary'] = sertraline_df['post_body'].apply(summarize_text)

In [40]:
fluoxetine_df['post_summary'] = fluoxetine_df['post_body'].apply(summarize_text)

In [41]:
ketamine_df['post_summary'] = ketamine_df['post_body'].apply(summarize_text)

In [42]:
citalopram_df['post_summary'] = citalopram_df['post_body'].apply(summarize_text)

In [43]:
print(bupropion_df['post_summary'])

0    Questioning if taking Wellbutrin XR at night i...
1    User concerned about antidepressants dulling c...
2    User's primary concern: Need for further study...
3    User feels hopeless and unable to pursue softw...
4    User is losing hope and seeking advice on what...
5    User is seeking advice on what to try next for...
Name: post_summary, dtype: object


In [44]:
# Define the API endpoint URL
url = "https://www.reddit.com/r/psilocybintherapy/top/.json?limit=100&t=year"

# Add user agent header to avoid 429 error
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Make the API request
response = requests.get(url, headers=headers)

# Check if the response was successful
if response.status_code != 200:
    raise ValueError("Failed to fetch posts: {}".format(response.status_code))

data = response.json()
posts_data = []
for post in data["data"]["children"]:
    post_title = post["data"]["title"]
    post_body = post["data"]["selftext"]
    user = post["data"]["author"]
    posts_data.append({"user": user, "post_title": post_title, "post_body": post_body})


In [45]:
df_psil= pd.DataFrame(posts_data)
df_psil.to_excel('C:/Users/ryan_/OneDrive/School/Pharma/coding/psil_data.xlsx', index=False)

In [47]:
df_psil[['pos', 'neg', 'neu', 'compound']] = df_psil['post_title'].apply(lambda x: pd.Series(get_sentiment_scores(x)))

                user                                         post_title  \
0    Ill-Pension-525  Janis Hughes is a stage IV cancer patient and ...   
1       bluemoodfood                          Hope this is helpful! ✨🍄💊   
2         HelpMePeez                    Welcome to r/PsilocybinTherapy!   
3  SubtleWindsOregon  New, peer-reviewed, mid-stage trial found that...   
4       Lady-vanilla  2nd psilocybin successful “self-directed thera...   

                                           post_body    pos    neg    neu  \
0                                                     0.000  0.292  0.708   
1                                                     0.666  0.000  0.334   
2  This sub was created a while ago and quickly f...  0.622  0.000  0.378   
3                                                     0.150  0.102  0.748   
4  I believe that we need to celebrate our succes...  0.432  0.000  0.568   

   compound  
0   -0.8934  
1    0.7177  
2    0.5093  
3    0.1779  
4    0.5859  


In [49]:
df_psil['post_summary'] = df_psil['post_body'].apply(summarize_text)

In [51]:
def primary_use(text):
    prompt = (
        f"I want you to act as natural language assistant specializing in pharmaceuticals. You will be given the body text of a reddit post, I want you to read the user's post and concisely summarize it to identify why the user is interested in psilocybin, and their experience if they have used it. Your summary must be under 20 words. Do not start the response with any introduction such as 'The user is', just cut to the summary. Please read the following post and provide your summary:\n\n{text}\n")

    # Make API call to OpenAI
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.1,
    )

    # Extract first 5 words from API response
    output = response.choices[0].text.strip().split()
    summary = ' '.join(output)

    # Return summary
    return summary

In [52]:
df_psil['primary_use'] = df_psil['post_body'].apply(primary_use)

In [54]:
# Define the API endpoint URL
url = "https://www.reddit.com/r/psychedelictherapy/top/.json?limit=100&t=year"

# Add user agent header to avoid 429 error
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Make the API request
response = requests.get(url, headers=headers)

# Check if the response was successful
if response.status_code != 200:
    raise ValueError("Failed to fetch posts: {}".format(response.status_code))

data = response.json()
posts_data = []
for post in data["data"]["children"]:
    post_title = post["data"]["title"]
    post_body = post["data"]["selftext"]
    user = post["data"]["author"]
    posts_data.append({"user": user, "post_title": post_title, "post_body": post_body})

In [55]:
df_psych= pd.DataFrame(posts_data)
df_psych.to_excel('C:/Users/ryan_/OneDrive/School/Pharma/coding/psych_data.xlsx', index=False)

In [57]:
df_psych[['pos', 'neg', 'neu', 'compound']] = df_psych['post_title'].apply(lambda x: pd.Series(get_sentiment_scores(x)))

            user                                         post_title  \
0  redditor01020  The governor of Maryland announced on Friday t...   
1        shallah  Australia will allow prescription MDMA and mag...   
2  redditor01020  Colorado voters have passed a ballot initiativ...   
3         mjobby  “Forgive yourself for not knowing better at th...   
4   Not_Not_Matt  Australia approves the therapeutic use of MDMA...   

                                           post_body    pos    neg    neu  \
0                                                     0.092  0.200  0.708   
1                                                     0.109  0.087  0.804   
2                                                     0.070  0.000  0.930   
3   \n\nJust liked the quote, felt it relevant he...  0.204  0.098  0.698   
4                                                     0.153  0.000  0.847   

   compound  
0   -0.7351  
1   -0.0772  
2    0.2732  
3    0.5565  
4    0.4019  


In [58]:
df_psych['drug_name_body'] = df_psych['post_body'].apply(retrieve_drugName)
df_psych['drug_name_title'] = df_psych['post_title'].apply(retrieve_drugName)

In [59]:
print(df_psych.head(20))

                   user                                         post_title  \
0         redditor01020  The governor of Maryland announced on Friday t...   
1               shallah  Australia will allow prescription MDMA and mag...   
2         redditor01020  Colorado voters have passed a ballot initiativ...   
3                mjobby  “Forgive yourself for not knowing better at th...   
4          Not_Not_Matt  Australia approves the therapeutic use of MDMA...   
5               shallah  MDMA-assisted therapy for PTSD show: MAPS PBC ...   
6         redditor01020  Cory Booker And Rand Paul File Bill To Resched...   
7            HaiyoHaiyo   Anyone else think this is fucked up advertising?   
8               shallah  Oregon to Offer Magic Mushrooms for Psilocybin...   
9   Affectionate-Ad6318  Clinical depression healed in a single self-gu...   
10              lobesey  Spanish charity offering free, legal, therapeu...   
11              shallah  In defiance of federal drug law, mushro

In [60]:
drug_count = {}

for index, row in df_psych.iterrows():
    drugs = row['drug_name_title'].split(", ")  # assuming drug names are separated by comma and space
    for drug in drugs:
        drug = drug.split(":")[-1].strip().lower()  # get the drug name after the colon (if any) and convert to lowercase
        if drug in drug_count:
            drug_count[drug] += 1
        else:
            drug_count[drug] = 1

In [61]:
ranked_psych = pd.DataFrame(drug_count.items(), columns=['drug_name_title', 'count']).sort_values(by='count', ascending=False)
print(ranked_psych.head(50))

                   drug_name_title  count
7                         not sure     34
3                              n/a     25
0                       psilocybin     19
2                         ketamine     10
1                             mdma      9
8                        not sure.      7
5                              lsd      2
4                         maps pbc      1
6                              ifs      1
9                        ayahuasca      1
10                         ecstasy      1
11                    talk therapy      1
12  psychedelic-assisted therapies      1
13         stellate ganglion block      1


In [62]:
ranked_psych['drug_name_title'].replace(['N/A', '', 'n/a', 'not sure.', 'not sure', 'psychedelic-assisted therapies','talk therapy'], pd.NaT, inplace=True)
ranked_psych= ranked_psych.dropna(subset=['drug_name_title'])

In [63]:
print(ranked_psych.head())

  drug_name_title  count
0      psilocybin     19
2        ketamine     10
1            mdma      9
5             lsd      2
4        maps pbc      1


In [64]:
total_count = ranked_psych['count'].sum()
ranked_psych['share'] = ranked_psych['count'] / total_count
print(ranked_psych.head(20))

            drug_name_title  count     share
0                psilocybin     19  0.422222
2                  ketamine     10  0.222222
1                      mdma      9  0.200000
5                       lsd      2  0.044444
4                  maps pbc      1  0.022222
6                       ifs      1  0.022222
9                 ayahuasca      1  0.022222
10                  ecstasy      1  0.022222
13  stellate ganglion block      1  0.022222


In [66]:
# create an ExcelWriter object and specify the file path
with pd.ExcelWriter('social_listening.xlsx') as writer:

    # write each dataframe to a different sheet ien the Excel file
    ranked_drugs.to_excel(writer, sheet_name='Drug Mention Shares')
    ranked_psych.to_excel(writer, sheet_name='Psychadelic Mention Shares')
    escitalopram_df.to_excel(writer, sheet_name='Escitalopram Summary', index=False)
    bupropion_df.to_excel(writer, sheet_name='Bupropion Summary', index=False)
    sertraline_df.to_excel(writer, sheet_name='Sertraline Summary')    
    fluoxetine_df.to_excel(writer, sheet_name='Fluoxetine Summary', index=False)
    citalopram_df.to_excel(writer, sheet_name='Citalopram Summary', index=False)
    ketamine_df.to_excel(writer, sheet_name='Ketamine Summary', index=False)
    df_filtered4.to_excel(writer, sheet_name='Antidepressant posts', index=False)
    df_psil.to_excel(writer, sheet_name='Psilocybin posts', index=False)

    # close the writer object
    writer.save()

C:\Users\ryan_\AppData\Local\Temp\ipykernel_39640\2383495805.py:17: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
